# State-level Census API calls


Link to the census API documentation: https://pypi.python.org/pypi/census

In [47]:
# Install these prior to running anything
# !pip install census
# !pip install us

### Import Libraries

In [74]:
from census import Census
from us import states
import csv
import pandas as pd
import numpy as np

import time
from __future__ import print_function

#Custom API Key below
c = Census("7fd39f36e6cc7f3548a80cc2c0c1fb6ca6ac44f1")

## Import Reference File and fill blank values

In this file, there are 2 fields with API field IDs:
- **Single_Field**: This is the ID for an attribute where the value can be pulled using just one field
- **Agg_Field**: This is a list of comma-delimited IDs that is used when an multiple fields need to be aggregated to provide the value we're seeking

In [75]:
field_ref = pd.read_excel('./Census_Field_Master.xls').fillna('NA')

## Function to pull state-level metrics

**Inputs**:
- A full state name
- A state's Federal Information Processing Standard code
- The year you'd like to pull data for

**NOTE**: I am using 5-year ACS data in the function below. This can be changed, if needed.

In [76]:
def state_demographics(state, state_fips, api_year):
    demo_dict={'State':state,'Year':api_year}
    
    for i in range(len(field_ref['Single_Field'])):
        tmp=0
        denom=0
        single_field_id = field_ref['Single_Field'][i]
        denom_id = field_ref['Denominator'][i]

        
        ### First, check to see if the value can be found with just 1 call to the API, 
        ### or if we need to aggregate the value with multiple API calls.
        
        if single_field_id == 'NA':
            ### If we need to compile multiple attributes to formulate the right value,
            ### we split the "Agg_Field" by  the comma, and make an API call for each field ID
            
            agg_fields = field_ref['Agg_Field'][i]
            agg_fields = agg_fields.split(',')
            for j in agg_fields:
                tmp += c.acs5.state(('NAME', j), state_fips, year=api_year)[0][j]
                
            if denom_id != 'NA':
                denom = c.acs5.state(('NAME', denom_id), state_fips, year=api_year)[0][denom_id]
                tmp = round((float(tmp)/denom),5)*100
                
            #add keys to dictionary to allow the final data frame to have
            # 1 row per state and 1 column per attribute
            demo_dict[str(field_ref['Description'][i])]= tmp
            
        ### If we don't need to aggregate multiple fields, we can just make a single call
        
        else:
            tmp = c.acs5.state(('NAME', single_field_id), state_fips, year=api_year)[0][single_field_id]
            if denom_id != 'NA':
                denom = c.acs5.state(('NAME', denom_id), state_fips, year=api_year)[0][denom_id]
                tmp = round((float(tmp)/denom),5)*100
                
                #add keys to dictionary to allow the final data frame to have
                # 1 row per state and 1 column per attribute
                demo_dict[str(field_ref['Description'][i])]= tmp

            
    ### return the data in the form of a list
    l=[]
    l.append(demo_dict)
    return l


### Loop through each US state and execute the function created above.

In [79]:
### Create a mapping dictionary to map a state name to the FIPS code
state_dict = states.mapping('name', 'fips')
agg_demo_list = []

state_progress=str()
start = time.time()

### Call the 'us.states' library for a list of all state names
for i in states.STATES:
    
    ### append each state's execution to a list
    ### to get the state FIPS code, I'm referencing the mapping dictionary created above
    agg_demo_list.extend(state_demographics(str(i),state_dict[str(i)],2015))
    
    ## 
    state_progress=state_progress+' '+str(i)
    print("Completed States: "+state_progress, end='\r')
    
    
### Convert the compiled list to a pandas dataframe for further processing
d = pd.DataFrame(agg_demo_list)

print("\nMinutes to complete: ",round((time.time()-start)/60,2))

Completed States:  Alabama Alaska Arizona Arkansas California Colorado Connecticut Delaware District of Columbia Florida Georgia Hawaii Idaho Illinois Indiana Iowa Kansas Kentucky Louisiana Maine Maryland Massachusetts Michigan Minnesota Mississippi Missouri Montana Nebraska Nevada New Hampshire New Jersey New Mexico New York North Carolina North Dakota Ohio Oklahoma Oregon Pennsylvania Rhode Island South Carolina South Dakota Tennessee Texas Utah Vermont Virginia Washington West Virginia Wisconsin Wyoming
Minutes to complete:  16.22


### Print the resulting data frame

In [80]:
d

,State,Year,age_0_17,age_18_29,age_30_39,age_40_49,age_50_59,age_60_plus,education_bachelors_or_higher_age_25_64,education_high_school_diploma_age_25_64,...,race_american_indian_and_alaska_native,race_asian,race_black,race_one_race,race_other,race_pacific_islander,race_two_or_more_races,race_white,with_health_insurance,without_health_insurance
0,Alabama,2015,23.019,16.439,12.435,13.163,13.899,21.045,24.738,29.909,...,0.494,1.234,26.426,98.310,1.264,0.050,1.690,68.841,87.343,12.657
1,Alaska,2015,25.652,19.292,13.598,12.674,14.266,14.518,27.901,28.208,...,13.815,5.853,3.412,91.579,1.264,1.206,8.421,66.030,81.820,18.180
2,Arizona,2015,24.350,16.724,12.766,12.589,12.513,21.059,27.637,23.489,...,4.437,3.013,4.239,96.810,6.487,0.188,3.190,78.446,84.964,15.036
3,Arkansas,2015,23.927,16.240,12.605,12.655,13.343,21.229,22.310,34.120,...,0.622,1.364,15.541,97.886,2.112,0.231,2.114,78.015,85.838,14.162
4,California,2015,23.878,17.842,13.815,13.608,13.069,17.787,32.043,20.342,...,0.747,13.695,5.896,95.485,12.948,0.391,4.515,61.807,85.298,14.702
5,Colorado,2015,23.525,17.242,14.169,13.370,13.669,18.026,39.110,20.678,...,0.947,2.907,4.050,96.524,4.250,0.146,3.476,84.224,87.684,12.316
6,Connecticut,2015,21.839,15.752,11.821,14.240,15.160,21.188,39.881,25.677,...,0.248,4.193,10.311,97.164,5.062,0.027,2.836,77.322,92.098,7.902
7,Delaware,2015,22.036,16.543,12.226,12.920,14.131,22.144,31.565,29.724,...,0.339,3.631,21.640,97.273,2.266,0.033,2.727,69.364,91.763,8.237
8,District of Columbia,2015,17.190,24.624,18.061,12.279,11.558,16.288,57.532,16.915,...,0.333,3.679,48.873,97.324,4.195,0.039,2.676,40.206,94.186,5.814
9,Florida,2015,20.570,15.496,12.063,13.230,13.860,24.781,28.131,28.551,...,0.278,2.591,16.141,97.597,2.510,0.056,2.403,76.020,82.030,17.970


### Save to xls

In [81]:
d.to_excel('./state_level_census.xls')

# Appendix

### Only needed if we want to test certain fields

In [24]:
#field_list = ['B27002_004E','B27002_007E','B27002_010E','B27002_013E','B27002_016E','B27002_019E','B27002_022E','B27002_025E','B27002_028E','B27002_032E','B27002_035E','B27002_038E','B27002_041E','B27002_044E','B27002_047E','B27002_050E','B27002_053E','B27002_056E']
#field_list = ['B27002_005E','B27002_008E','B27002_011E','B27002_014E','B27002_017E','B27002_020E','B27002_023E','B27002_026E','B27002_029E','B27002_033E','B27002_036E','B27002_039E','B27002_042E','B27002_045E','B27002_048E','B27002_051E','B27002_054E','B27002_057E']
field_list=['B23006_001E']

tmp=0

for i in field_list:
    tmp += c.acs5.state(('NAME', i), states.AL.fips, year=2015)[0][i]
    
    
print (tmp)

2520025.0


### If we want to have multiple rows per state

In [45]:
def state_demographics(state, state_fips, api_year):
    demo_list=[]
    
    for i in range(len(field_ref['Single_Field'])):
        tmp=0
        denom=0
        single_field_id = field_ref['Single_Field'][i]
        denom_id = field_ref['Denominator'][i]

        
        ### First, check to see if the value can be found with just 1 call to the API, 
        ### or if we need to aggregate the value with multiple API calls.
        
        if single_field_id == 'NA':
            ### If we need to compile multiple attributes to formulate the right value,
            ### we split the "Agg_Field" by  the comma, and make an API call for each field ID
            
            agg_fields = field_ref['Agg_Field'][i]
            agg_fields = agg_fields.split(',')
            for j in agg_fields:
                tmp += c.acs5.state(('NAME', j), state_fips, year=api_year)[0][j]
                
            if denom_id != 'NA':
                denom = c.acs5.state(('NAME', denom_id), state_fips, year=api_year)[0][denom_id]
                tmp = round((float(tmp)/denom),5)*100
            demo_list.append({'State':state,'Attribute':field_ref['Description'][i],'Value': tmp,'Year':api_year})
            
        ### If we don't need to aggregate multiple fields, we can just make a single call
        
        else:
            tmp = c.acs5.state(('NAME', single_field_id), state_fips, year=api_year)[0][single_field_id]
            if denom_id != 'NA':
                denom = c.acs5.state(('NAME', denom_id), state_fips, year=api_year)[0][denom_id]
                tmp = round((float(tmp)/denom),5)*100
            demo_list.append({'State':state,'Attribute':field_ref['Description'][i],'Value': tmp,'Year':api_year})
            
    ### return the data in the form of a list
    
    return demo_list
